In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ratingstxt/ratings.txt
/kaggle/input/goodbooks-10k/book_tags.csv
/kaggle/input/goodbooks-10k/to_read.csv
/kaggle/input/goodbooks-10k/tags.csv
/kaggle/input/goodbooks-10k/books.csv
/kaggle/input/goodbooks-10k/sample_book.xml
/kaggle/input/goodbooks-10k/ratings.csv


In [2]:
import urllib
import io
import zipfile
import surprise

# Download zip file
# tmpFile = urllib.request.urlopen('https://librec.net/datasets/fimltrust.zip')
# Unzip file
# tmpFile = zipfile.zipFile(io.BytesIO(tmpFile.read()))
# Open desired data file as pandas dataframe, close zipfile
# dataset = pd.read_table(io.BytesIO(tmpFile.read('ratings.txt')), sep = ' ', names = ['uid', 'iid', 'rating'])
dataset = pd.read_csv('/kaggle/input/ratingstxt/ratings.txt', sep = ' ', names = ['uid', 'iid', 'rating'])
# tmpFile.close()

dataset.head()

,uid,iid,rating
0,1,1,2.0
1,1,2,4.0
2,1,3,3.5
3,1,4,3.0
4,1,5,4.0


In [3]:
lower_rating = dataset['rating'].min()
upper_rating = dataset['rating'].max()
print('Review range: {0} to {1}'.format(lower_rating, upper_rating))

Review range: 0.5 to 4.0


In [4]:
reader = surprise.Reader(rating_scale = (0.5,4))
data = surprise.Dataset.load_from_df(dataset, reader)

In [5]:
alg = surprise.SVDpp()
output = alg.fit(data.build_full_trainset())

In [6]:
# the uids and iids should be set as strings
pred = alg.predict(uid = '50', iid = '52')
score = pred.est
print(score)

3.0028030537791928


In [7]:
# Get a list of all movie ids
iids = dataset['iid'].unique()
# Get a list of iids that uid 50 has rated
iids50 = dataset.loc[dataset['uid'] == 50, 'iid']
# Remove the iids that uid 50 has rated from the list of all movie ids
iids_to_pred = np.setdiff1d(iids, iids50)
# np.setdiff1d -> Return the unique values in ar1 that are not in ar2

In [8]:
testset = [[50, iid, 4.] for iid in iids_to_pred]
predictions = alg.test(testset)
predictions[0]

Prediction(uid=50, iid=14, r_ui=4.0, est=2.9982213132502, details={'was_impossible': False})

In [9]:
pred_ratings = np.array([pred.est for pred in predictions])
# Find the indec of the maximum predicted rating
i_max = pred_ratings.argmax()
# Use this to find the corresponding iid to recommend
iid = iids_to_pred[i_max]
print('Top item for user 50 has iid {0} with predicted rating {1}'.format(iid, pred_ratings[i_max]))

Top item for user 50 has iid 189 with predicted rating 4.0


In [10]:
pred_ratings = np.array([pred.est for pred in predictions])
# Find the indec of the maximum predicted rating
i_max = pred_ratings.argpartition(pred_ratings, 0, axis = None)[0:]
# Use this to find the corresponding iid to recommend
iid = iids_to_pred[i_max]
print('Top item for user 50 has iid {0} with predicted rating {1}'.format(iid, pred_ratings[i_max]))

TypeError: Argument given by name ('axis') and position (2)

In [11]:
param_grid = {'lr_all' : [.001, .01], 'reg_all' : [.1, .5]}
gs = surprise.model_selection.GridSearchCV(surprise.SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)
# Print combination of parameters that gave best RMSE score
print(gs.best_params['rmse'])

{'lr_all': 0.01, 'reg_all': 0.1}


In [12]:
alg = surprise.SVDpp(lr_all = 0.01) # parameter choices can be added here
output = surprise.model_selection.cross_validate(alg, data, verbose = True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8047  0.8124  0.8012  0.7989  0.8019  0.8038  0.0047  
MAE (testset)     0.6172  0.6193  0.6119  0.6139  0.6094  0.6144  0.0036  
Fit time          18.40   18.59   18.34   18.43   18.39   18.43   0.08    
Test time         0.34    0.30    0.30    0.32    0.38    0.33    0.03    
